## Add a garment aligned with canonical SMPL(-X) body


To add a new garment, you should have an `.obj` file with the garment mesh that is **aligned with the canonical body model** (zero-pose and zero-shape) you want to use.

*If you only have garment geometry aligned with an arbitrary SMPL(-X) body, see the section for this below.*

We currently support [SMPL](https://smpl.is.tue.mpg.de/) and [SMPL-X](https://smpl-x.is.tue.mpg.de/) body models.

Note: you will not be able to run LBS-initialized simulation with a body model that is different from the one you specify in this step.

First, create a .pkl file with your garment data using `add_garment_to_garments_dict` function. 

We call such .pkl files "garment dicts". They are the key entity used to store garment information for ContourCraft.

<details>

<summary>
Click for more details
</summary>

It builds a dictionary for the garment that contains:
* `rest_pos`: \[Nx3\], positions of the vertices in canonical pose that are aligned to zero- pose and shape SMPL body.
* `faces`: \[Fx3\], triplets of node indices that constitute each face
* `node_type` \[Nx1\], node type labels (`0` for regular, `3` for "pinned"). By default, all nodes are regular, we show how to add "pinned nodes" later in this notebook
* `lbs` dictionary with shape- and pose- blendshapes and skinning weights for the garment, sampled from SMPL(-X) model
* `center` and `coarse_edges` info on long-range (coarse) edges used to build a hiererchical graph of the garment.

To be able to start simulation from an arbitrary pose, we use linear blend-skinning (LBS) to initialize the garment geometry in the first frame. For each garment node we sample pose- and shape-blendshapes and skinning weights from the closest SMPL(-X) node in canonical pose.

However, for loose garments such approach may result in overly-stretched triangles. Therefore, we use the approach introduced in [\[Santesteban et al. 2021\]](http://mslab.es/projects/SelfSupervisedGarmentCollisions/) and average skinning weights and blendshapes over many randomly sampled SMPL(-X) nodes around the given garment node.

The parameter `n_samples_lbs` controls the number of random samples to use. We recommend setting it to 0 for tight-fitting garments (shirts, pants) and to 1000 for loose ones (skirts, dresses).

</details>

In [ ]:
import os
from pathlib import Path
from utils.mesh_creation import GarmentCreator, add_pinned_verts
from utils.defaults import DEFAULTS


garment_obj_path = os.path.join(DEFAULTS.aux_data, 'garment_meshes', 'smplx', 'cindy_020_combined.obj')

body_models_root = os.path.join(DEFAULTS.aux_data, 'body_models')

# Model type, either 'smpl' or 'smplx'
model_type = 'smplx'

# gender, either 'male`, `female` or `neutral`
gender = 'female'

garment_dicts_dir =  Path(DEFAULTS.aux_data) / 'garment_dicts' / model_type
garment_name = 'cindy_020_combined_test'

# Use approximate_center=True to create temlate faster. In the original paper code it was False
gc = GarmentCreator(garment_dicts_dir, body_models_root, model_type, gender, 
                    n_samples_lbs=0, verbose=True, coarse=True, approximate_center=True)
gc.add_garment(garment_obj_path, garment_name)

# Now garment 'cindy_020_combined_test' is added to the garments_dict.pkl file and can be used in furter steps


### Add pinned vertices

For some gaments, it is necessary to fix positions for a subset of nodes relative to the body. For example, fix the top ring of a skirt or pants to prevent it from falling off the body.

To label a set of garment nodes as "pinned", you need to use `add_pinned_verts` function and provide it with the list of node indices that you want to pin.


<details>

<summary>
How to get vertex indices in Blender
</summary>

One easy way of getting indices for a set of nodes, is by using [Blender](https://www.blender.org/). 

1. Open it, import the garment from the `.obj` file. 
2. Then in `Layout` tab press `Tab` to go into the editing mode. 
3. Select all vertices you want to pin. 
4. Then, go into `Scripting` tab and execute the following piece of code there.

```python
import bpy
import bmesh

obj = bpy.context.active_object
bm = bmesh.from_edit_mesh(obj.data)    
obj = bpy.context.active_object; bm = bmesh.from_edit_mesh(obj.data)    ; selected = [i.index for i in bm.verts if i.select == True]; print(selected)

```


5. You will get a list of indices for the selected nodes.


</details>

Below are pinned indices for the garment `cindy_020_combined`, replace them with yours

In [ ]:
pinned_indices = \
[   26,    27,    28,    36,    47,    56,    68,    69,    79,
          93,    94,   103,   121,   130,   151,   161,   185,   197,
         222,   237,   262,   279,   280,   307,   326,   353,   372,
         402,   422,   455,   456,   478,   513,   538,   576,   603,
         641,   670,   706,   738,   778,   812,   855,   891,   936,
         973,  1020,  1061,  1109,  1151,  1200,  1243,  1293,  1339,
        1390,  1439,  1492,  1543,  1597,  1651,  1706,  1761,  1821,
        1878,  1936,  1994,  2052,  2112,  2176,  2238,  2306,  2369,
        2439,  2508,  2576,  2644,  2713,  2719,  2784,  2858,  2926,
        3001,  3071,  3148,  3218,  3296,  3368,  3448,  3522,  3607,
        3687,  3770,  3850,  3934,  4016,  4102,  4187,  4269,  6567,
        6626,  7071,  7368,  7577,  7804,  7920,  8040,  8460,  8550,
        8632,  8992,  9065,  9137, 11694, 11695, 11700, 11706, 11712,
       11715, 11726, 11731, 11735, 11739, 11743, 11747, 11755, 11761,
       11770, 11779, 11792, 11797, 11802, 11814, 11819, 11824, 11829,
       11851, 11857]

gc.add_pinned_verts(garment_name, pinned_indices)

## Adding garments aligned with an arbitrary SMPL(-X) pose

You can also import a garment aligned with an arbitrary SMPL(-X) body. 

To do this you will need:
* a .pkl file containing SMPL(-X) parameters (details below), and
* an .obj file with the garment mesh which is aligned with this body

The process runs in two steps:
1. We un-pose and un-shape the garment using the linear-blend skinning weights from the body model. This step alignes the garment with the canonical body geometry, but may introduce very unrealistic geometries.
2. We simulate the garment over the canonical body to relax the artifacts. Here (and in subsequent simulations) we use the original garment geometry as the "resting" one. After the relaxation this garment is imported to a .pkl garment dict with the usual procedure.

We provide the example garment geometries and the body poses for both SMPL and SMPL-X models in `DEFAULTS.data_root/examples/unpose`.

The garment geometries are stored in simple .obj files.

The pose sequences are stored as .pkl files that contain dictionaries with the SMPL(-X) parameters. Below you can see the names and the shapes of the parameters in these files:

<details>
  <summary>for SMPL</summary>


```python
"betas": (1, 10)
"transl": (1, 3)
"global_orient": (1, 3)
"body_pose": (1, 69)
```

</details>

<details>
  <summary>for SMPL-X</summary>


```python
"betas": (1, 10)
"transl": (1, 3)
"global_orient": (1, 3)
"body_pose": (1, 63)
"jaw_pose": (1, 3)
"left_hand_pose": (1, 45)
"right_hand_pose": (1, 45)
"leye_pose": (1, 3)
"reye_pose": (1, 3)
```

</details>

You can use functions `create_smpl_pose_file` and `create_smplx_pose_file` in `utils/mesh_creation.py` to create such .pkl files from a frame of an AMASS pose sequence.

In [ ]:
from utils.mesh_creation import GarmentCreator
import os
from pathlib import Path
from utils.defaults import DEFAULTS

body_models_root = Path(DEFAULTS.aux_data) /  'body_models'




# # ================  Test for SMPL-X ============
model_type = 'smplx'
gender = 'male'

# Directory, where garment dictionaries are stored
garment_dicts_dir =  Path(DEFAULTS.aux_data) / 'garment_dicts' / 'smplx'

# Path to the garment object file
garment_obj_path = Path(DEFAULTS.data_root) / 'examples' / 'unpose' / 'smplx_garment.obj'

# Path to the body parameters file
body_params_file = Path(DEFAULTS.data_root) / 'examples' / 'unpose' / 'smplx_posed.pkl'
# # ============================================

# ================  Test for SMPL ============
# model_type = 'smpl'
# gender = 'male'
# garment_dicts_dir =  Path(DEFAULTS.aux_data) / 'garment_dicts' / 'smpl'
# garment_obj_path = Path(DEFAULTS.data_root) / 'examples' / 'unpose' / 'smpl_garment.obj'
# body_params_file = Path(DEFAULTS.data_root) / 'examples' / 'unpose' / 'smpl_posed.pkl'
# ============================================


# Name of the garment, the garment dict will be saved as {garment_dicts_dir}/{garment_name}.pkl
garment_name = 'unposed_garment2'

# Path to the ContourCraft checkpoint
models_dir = Path(DEFAULTS.data_root) / 'trained_models'
checkpoint_path = models_dir / 'contourcraft.pth'

gc = GarmentCreator(garment_dicts_dir, body_models_root, model_type, gender, 
                    n_samples_lbs=0, verbose=True, coarse=True, approximate_center=True)

relaxation_trajectory = gc.add_posed_garment(garment_obj_path, garment_name, body_params_file, 
                                             checkpoint_path, n_relaxation_steps=30)


Now you can also save the relaxation trajectory and see in in AITViewer for debugging purposes:

In [ ]:
from utils.io import pickle_dump
out_trajectory_path = Path(DEFAULTS.data_root) / 'temp' / 'relax_trajectory.pkl'
pickle_dump(relaxation_trajectory, out_trajectory_path)
print('Relaxation trajectory saved to', out_trajectory_path)

To see it in AITviewer, run:
```
python utils/show.py rollout_path=PATH_TO_SEQUENCE
```